## Import Library

In [5]:
import numpy as np
import pandas as pd
import csv
import mysql.connector
import mysql
from mysql.connector import Error

import threading

## Create connection to database

In [6]:
def create_connection(host, user, password, database):
    try:
        connection = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )
        if connection.is_connected():
            print(f'Connected to MySQL database: {database}')
            return connection
    except Error as e:
        print(f'Error: {e}')
        return None

In [7]:
db_host = 'localhost'
db_user = 'root'
db_password = 'your_password'
db_name = 'iranketab_db'

In [8]:
connection = create_connection(db_host, db_user, db_password, db_name)
cursor = connection.cursor()

Connected to MySQL database: iranketab_db


## Import publisher to publisher_name table

In [29]:
df_publisher = pd.read_csv('Book.csv', usecols=['Publisher'])
df_publisher = df_publisher.drop_duplicates()
df_publisher = df_publisher.reset_index(drop=True)
df_publisher

,Publisher
0,نشر چشمه
1,ندای معاصر (زرین کلک)
2,نیلوفر
3,مروارید
4,مجید (به سخن)
...,...
2036,نبش استور
2037,نفحات
2038,نیکوروش
2039,زندگی نو


In [32]:
table_name = 'publisher'

for _, row in df_publisher.iterrows():
        insert_query = f"INSERT INTO {table_name} (publisher_name) VALUES ('{row['Publisher']}')"
        cursor.execute(insert_query)

connection.commit()


Data inserted into table: publisher


## Import type of print to type_of_print table

In [5]:
df_type_of_print = pd.read_csv('Book.csv', usecols=['type_of_print'])
df_type_of_print = df_type_of_print.drop_duplicates()
df_type_of_print = df_type_of_print.reset_index(drop=True)
df_type_of_print

,type_of_print
0,جلد سخت
1,شومیز
2,زرکوب
3,جلد نرم
4,سلفونی
5,Not Define
6,طرح چرم با قاب کشویی
7,چرم طبیعی برجسته
8,جلد سخت همراه با قاب
9,چرم با قاب بازشو


In [6]:
table_name = 'type_of_print'

for _, row in df_type_of_print.iterrows():
        insert_query = f"INSERT INTO {table_name} (type_of_print) VALUES ('{row['type_of_print']}')"
        cursor.execute(insert_query)
        
connection.commit()

## Import summary to summery table

In [7]:
df_summery = pd.read_csv('Book.csv', usecols=['Description'])
df_summery = df_summery.drop_duplicates()
df_summery = df_summery.reset_index(drop=True)
df_summery

,Description
0,جسپر دین، بیشتر عمرش نمی توانست تصمیم بگیرد چه...
1,امیر، پسر یک تاجر ثروتمند کابلی و از قبیله ی ص...
2,داستان های مهیج هری پاتر، جادوگر جوانی که تحت ...
3,رمان نام من سرخ، معمایی شرورانه، عاشقانه ای جذ...
4,این رمان نفس گیر و هیجان انگیز، با تصادفی شبان...
...,...
70407,خدای سبحان احاطه مطلق وجودی دارد و این احاطه م...
70408,دخترهای گلم و پسرهای نازنینم شاهنامک میخواد خی...
70409,خواهران روایتی است از دو زن که از تبعید و رویا...
70410,در حالی که برخی بازی را بی‌ارزش و بی‌نتیجه و ح...


In [8]:
table_name = 'summery'

for _, row in df_summery.iterrows():
    # Use parameterized queries to safely insert data
    insert_query = "INSERT INTO {} (summary_text) VALUES (%s)".format(table_name)
    cursor.execute(insert_query, (row['Description'],))

connection.commit()

## Import size to size table

In [9]:
df_size = pd.read_csv('Book.csv', usecols=['size'])
df_size = df_size.drop_duplicates()
df_size = df_size.reset_index(drop=True)
df_size

,size
0,رقعی
1,وزیری
2,جیبی
3,پالتویی
4,Not Define
5,رحلی
6,خشتی
7,سلطانی
8,بیاضی
9,رحلی کوچک


In [10]:
table_name = 'size'

for _, row in df_size.iterrows():
    # Use parameterized queries to safely insert data
    insert_query = "INSERT INTO {} (size_name) VALUES (%s)".format(table_name)
    cursor.execute(insert_query, (row['size'],))

connection.commit()

## Import category to category table

In [51]:
df_category = pd.read_csv('Book.csv', usecols=['Category'])
df_category = df_category.drop_duplicates()
df_category['Category'] = df_category['Category'].str.split(',')
new_df = pd.DataFrame(columns = ['Category'])
new_df['Category'] = pd.concat([pd.Series(row['Category']).explode() for _, row in df_category.iterrows()], ignore_index=True)
new_df['Category'] = new_df['Category'].str.replace('[', '').str.replace(']', '').str.replace("'",'')
new_df = new_df.drop_duplicates()
new_df = new_df.dropna()
new_df


,Category
0,جایزه ی ان اس دبلیو پرایمر
1,ادبیات استرالیا
2,ادبیات داستانی
3,داستان فلسفی
4,ادبیات معاصر
...,...
279455,پرفروش ترین های چاپ 1402
281802,رنگ آمیزی
283797,الهیات
284928,جایزه ی آگوست(ادبی سوئد)


In [52]:
table_name = 'category'

for _, row in new_df.iterrows():
    # Use parameterized queries to safely insert data
    insert_query = "INSERT INTO {} (category_name) VALUES (%s)".format(table_name)
    cursor.execute(insert_query, (row['Category'],))

connection.commit()

## Import persons to person table

In [121]:
df_translator = pd.read_csv('Book.csv', usecols=['translator'])
df_translator = df_translator.drop_duplicates()
df_translator['translator'] = df_translator['translator'].str.split(',')
new_df = pd.DataFrame(columns = ['person'])
new_df['person'] = pd.concat([pd.Series(row['translator']).explode() for _, row in df_translator.iterrows()], ignore_index=True)
new_df['person'] = new_df['person'].str.replace('[', '').str.replace(']', '').str.replace("'",'')
new_df = new_df.drop_duplicates()
new_df = new_df.dropna()
new_df

,person
0,پیمان خاکسار
1,گلناز سهرابی
2,مهدی غبرایی
3,پریسا سلیمان زاده
4,زیبا گنجی
...,...
19395,جمال الدین اکرمی
19396,نویدا هادیان
19397,نوشین شعبانی
19399,زهرا سجادی


In [122]:
df_author = pd.DataFrame(columns=['person'])
df_author['person'] = pd.read_csv('Book.csv', usecols=['Writer'])
df_person = pd.concat([df_author, new_df], axis=0)
df_person = df_person.drop_duplicates()
df_person = df_person.dropna()
df_person = df_person.reset_index(drop=True)
df_person

,person
0,استیو تولتز
1,خالد حسینی
2,جی کی رولینگ
3,اورهان پاموک
4,پاتریک مودیانو
...,...
51680,یلدا واشقانی فراهانی
51681,مسعود نقاش زاده
51682,نویدا هادیان
51683,زهرا سجادی


In [128]:
table_name = 'person'

for _, row in df_person.iterrows():
    # Use parameterized queries to safely insert data
    insert_query = "INSERT INTO {} (person_name) VALUES (%s)".format(table_name)
    cursor.execute(insert_query, (row['person'],))

connection.commit()

## Import data to book table

### Cleaning inventory column

In [75]:
df_clean = pd.read_csv('Book.csv')
df_clean = df_clean.reset_index(drop=True)
df_clean.replace('موجود', 1, inplace=True)
df_clean.replace('ناموجود', 0, inplace=True)
df_clean.replace('Not Define', 0, inplace=True)
df_clean.replace('Unknown', np.NaN, inplace=True)
df_clean.replace('unknown', np.NaN, inplace=True)
df_clean['page_count'].replace(np.NaN, 0, inplace=True)
df_clean['gregorian_publish_year'] = df_clean['gregorian_publish_year'].astype('object')
df_clean['solar_publish_year'] = df_clean['solar_publish_year'].astype('object')
df_clean.to_csv('Book.csv', index=False)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103840 entries, 0 to 103839
Data columns (total 23 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   book_id                 103840 non-null  int64  
 1   print_series            103840 non-null  int64  
 2   size                    103840 non-null  object 
 3   type_of_print           103840 non-null  object 
 4   translator              55191 non-null   object 
 5   shabak                  103833 non-null  object 
 6   gregorian_publish_year  51349 non-null   object 
 7   solar_publish_year      103207 non-null  object 
 8   page_count              103840 non-null  int64  
 9   Persian Title           103840 non-null  object 
 10  English Title           103014 non-null  object 
 11  Off                     103840 non-null  int64  
 12  Rate                    103840 non-null  float64
 13  Break Price             103840 non-null  object 
 14  Special_Price       

In [55]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103840 entries, 0 to 103839
Data columns (total 23 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   book_id                 103840 non-null  int64  
 1   print_series            103840 non-null  int64  
 2   size                    103840 non-null  object 
 3   type_of_print           103840 non-null  object 
 4   translator              55191 non-null   object 
 5   shabak                  103833 non-null  object 
 6   gregorian_publish_year  51349 non-null   float64
 7   solar_publish_year      103207 non-null  float64
 8   page_count              103532 non-null  object 
 9   Persian Title           103840 non-null  object 
 10  English Title           103014 non-null  object 
 11  Off                     103840 non-null  int64  
 12  Rate                    103840 non-null  float64
 13  Break Price             103840 non-null  int64  
 14  Special_Price       

In [120]:
connection = create_connection(db_host, db_user, db_password, db_name)
cursor = connection.cursor(buffered=True)
try:
    with open('Book.csv', 'r') as csvfile:
        csvreader = csv.DictReader(csvfile)
        for row in csvreader:
            persian_title = row['Persian Title']
            english_title = row['English Title']
            isbn = row['shabak']
            book_code = row['book_id']
            release_date_sh = row['solar_publish_year']
            release_date_gc = row['gregorian_publish_year']
            rating = row['Rate']
            page_count = row['page_count']
            print_series = row['print_series']
            inventory = row['Exist']
            original_price = row['Break Price']
            discounted_price = row['Special_Price']
            discount_percentage = row['Off']
            size = row['size']
            summery = row['Description']
            type_of_print = row['type_of_print']
            publisher_name = row['Publisher']

            if not connection.is_connected():
                connection.connect()

            cursor.execute("SELECT id FROM publisher WHERE publisher_name = %s", (publisher_name,))
            result = cursor.fetchone()
            if result is not None:
                publisher_id = result[0]
            else:
                publisher_id = None

            cursor.execute("SELECT id FROM size WHERE size_name = %s", (size,))
            result1 = cursor.fetchone()

            if result1 is not None:
                size_id = result1[0]
            else:
                size_id = None

            cursor.execute("SELECT id FROM summery WHERE summary_text = %s", (summery,))
            result2 = cursor.fetchone()
            
            if result2 is not None:
                summery_id = result2[0]
            else:
                summery_id = None

            cursor.execute("SELECT id FROM type_of_print WHERE type_of_print = %s", (type_of_print,))
            result3 = cursor.fetchone()
            
            if result3 is not None:
                type_of_print_id = result3[0]
            else:
                type_of_print_id = None

            cursor.execute("INSERT INTO Book (product_name, product_name_eng, isbn, book_code, release_date_sh, release_date_gc, rating, book_pages,print_series, inventory, original_price, discount_price, discount_percent, size_id, summery_id, type_of_print_id, publisher_id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", (persian_title, english_title, isbn, book_code, release_date_sh, release_date_gc, rating, page_count, print_series, inventory, original_price, discounted_price, discount_percentage, size_id, summery_id, type_of_print_id, publisher_id))

            connection.commit()

except Exception as e:
    print(f"Error: {str(e)}")

finally:
    cursor.close()
    connection.close()


Connected to MySQL database: iranketab_db


## Import data to book_category table


In [145]:
connection = create_connection(db_host, db_user, db_password, db_name)
cursor = connection.cursor(buffered=True)
try:
    with open('Book.csv', 'r') as csvfile:
        csvreader = csv.DictReader(csvfile)
        for row in csvreader:
            book_name = row['Persian Title']
            cat = row['Category']
            category = category.split(',')
            for cat in category:
                cursor.execute("SELECT id FROM category WHERE category_name = %s", (cat,))
                result = cursor.fetchone()
                if result is not None:
                    category_id = result[0]
                else:
                    category_id = None
                    
            for book_name in book_code:
                cursor.execute("SELECT book_id FROM Book WHERE product_name = %s", (book_id,))
                result = cursor.fetchone()
                if result is not None:
                    book_id = result[0]
                else:
                    book_id = None
                
                cursor.execute("INSERT INTO book_category (book_id, category_id) VALUES (%s, %s)", (book_id, category_id))
                connection.commit()

except Exception as e:
    print(f"Error: {str(e)}")

finally:
    cursor.close()
    connection.close()

Connected to MySQL database: iranketab_db


KeyboardInterrupt: 

In [184]:
df = pd.read_csv('Book.csv', usecols=['Persian Title', 'Category'])

In [223]:
df = pd.read_csv('Book.csv', usecols=['Persian Title', 'Category'])

# Split the 'Category' column by comma and explode
df['Category'] = df['Category'].str.split(',')
df = df.explode('Category')

# Remove extra characters from the 'Category' column
df['Category'] = df['Category'].str.replace('[', '').str.replace(']', '').str.replace("'", '')

# Reset the index of the DataFrame
df.reset_index(drop=True, inplace=True)


df.to_csv('category.csv', index=False)



,Persian Title,Category
0,کتاب جزء از کل,جایزه ی ان اس دبلیو پرایمر
1,کتاب جزء از کل,ادبیات استرالیا
2,کتاب جزء از کل,ادبیات داستانی
3,کتاب جزء از کل,داستان فلسفی
4,کتاب جزء از کل,ادبیات معاصر
...,...,...
580976,کتاب روان شناسی بازی,روانشناسی کودک
580977,کتاب فرمانروایی هخامنشیان بر مصر,دهه 2010 میلادی
580978,کتاب فرمانروایی هخامنشیان بر مصر,تاریخی
580979,کتاب فرمانروایی هخامنشیان بر مصر,ادبیات ایران


In [245]:
connection = create_connection(db_host, db_user, db_password, db_name)
cursor = connection.cursor(buffered=True)

with open('category.csv', 'r') as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        title = row['Persian Title']
        cat1 = row['Category']
        categories = cat1.split(',')  # Split the category string into a list
        
        for cat in categories:
            cursor.execute("SELECT id FROM category WHERE category_name = %s", (cat.strip(),))  # Use strip() to remove any leading/trailing spaces
            result = cursor.fetchone()
            if result is not None:  # Check if a result was found
                category_id = result[0]
            else:
                category_id = None
                continue
        
        cursor.execute("SELECT book_id FROM book WHERE product_name = %s", (title,))
        result = cursor.fetchone()
        if result is not None:  # Check if a result was found
            book_id = result[0]
        else:
            book_id = None
            continue

        cursor.execute("INSERT INTO book_category (book_id, category_id) VALUES (%s, %s)", (book_id, category_id))
        connection.commit()

connection.close()


Connected to MySQL database: iranketab_db


## Import data to book_author table

In [249]:
df_author = pd.read_csv('Book.csv', usecols=['Persian Title','Writer'])
df_author['Writer'] = df['Writer'].str.split(',')
df_author = df.explode('Writer')

# Remove extra characters from the 'Category' column
df_author['Writer'] = df['Writer'].str.replace('[', '').str.replace(']', '').str.replace("'", '')

# Reset the index of the DataFrame
df_author.reset_index(drop=True, inplace=True)
df.to_csv('author.csv', index=False)


In [250]:
connection = create_connection(db_host, db_user, db_password, db_name)
cursor = connection.cursor(buffered=True)

with open('author.csv', 'r') as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        title = row['Persian Title']
        author1 = row['Writer']
        authors = author1.split(',')  # Split the category string into a list
        
        for author in authors:
            cursor.execute("SELECT id FROM person WHERE person_name = %s", (author.strip(),))  # Use strip() to remove any leading/trailing spaces
            result = cursor.fetchone()
            if result is not None:  # Check if a result was found
                author_id = result[0]
            else:
                author_id = None
                continue
        
        cursor.execute("SELECT book_id FROM book WHERE product_name = %s", (title,))
        result = cursor.fetchone()
        if result is not None:  # Check if a result was found
            book_id = result[0]
        else:
            book_id = None
            continue

        cursor.execute("INSERT INTO book_author (book_id, person_id) VALUES (%s, %s)", (book_id, author_id))
        connection.commit()

Connected to MySQL database: iranketab_db


## Import data to book_translator table

In [259]:
df_translator = pd.read_csv('Book.csv', usecols=['Persian Title', 'translator'])
df_translator

,translator,Persian Title
0,['پیمان خاکسار'],کتاب جزء از کل
1,['پیمان خاکسار'],جزء از کل (مجموعه برگ و نوا)
2,['گلناز سهرابی'],جزء از کل
3,['گلناز سهرابی'],جزء از کل
4,['مهدی غبرایی'],کتاب بادبادک باز
...,...,...
103835,NaN,کتاب اسرار عرفانی نماز
103836,NaN,کتاب شاهنامک
103837,['محبوبه میرفندرسکی'],کتاب خواهران
103838,NaN,کتاب روان شناسی بازی


In [262]:
df_translator['translator'] = df_translator['translator'].str.split(',')
df_translator = df_translator.explode('translator')

# Remove extra characters from the 'Category' column
df_translator['translator'] = df_translator['translator'].str.replace('[', '').str.replace(']', '').str.replace("'", '')

# Reset the index of the DataFrame
df_translator.reset_index(drop=True, inplace=True)
df_translator.to_csv('translator.csv', index=False)

In [263]:
connection = create_connection(db_host, db_user, db_password, db_name)
cursor = connection.cursor(buffered=True)

with open('translator.csv', 'r') as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        title = row['Persian Title']
        translator1 = row['translator']
        translators = translator1.split(',')  # Split the category string into a list
        
        for translator in translators:
            cursor.execute("SELECT id FROM person WHERE person_name = %s", (translator.strip(),))  # Use strip() to remove any leading/trailing spaces
            result = cursor.fetchone()
            if result is not None:  # Check if a result was found
                translator_id = result[0]
            else:
                translator_id = None
                continue
        
        cursor.execute("SELECT book_id FROM book WHERE product_name = %s", (title,))
        result = cursor.fetchone()
        if result is not None:  # Check if a result was found
            book_id = result[0]
        else:
            book_id = None
            continue

        cursor.execute("INSERT INTO book_translator (book_id, person_id) VALUES (%s, %s)", (book_id, translator_id))
        connection.commit()

Connected to MySQL database: iranketab_db


## Debuging

In [9]:
connection = create_connection(db_host, db_user, db_password, db_name)
cursor = connection.cursor()

cursor.execute("UPDATE book SET release_date_gc = null WHERE release_date_gc > 2023")
connection.commit()

Connected to MySQL database: iranketab_db


In [10]:
connection = create_connection(db_host, db_user, db_password, db_name)
cursor = connection.cursor()

cursor.execute("UPDATE book SET release_date_sh = null WHERE release_date_sh > 1402")
connection.commit()

Connected to MySQL database: iranketab_db


In [13]:
connection = create_connection(db_host, db_user, db_password, db_name)
cursor = connection.cursor()

cursor.execute("UPDATE book SET book_pages = book_pages / 10 where book_id Between 51113 AND 68116;")
connection.commit()

Connected to MySQL database: iranketab_db
